In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
# Input data
nodes = {'Well 1', 'Well 2', 'Mobile', 'Galveston', 'NY', 'LA'}
connections = { 
            ('Well 1', 'Mobile'), ('Well 1', 'Galveston'), ('Well 1', 'NY'), ('Well 1','LA'),
            ('Well 2', 'Mobile'), ('Well 2', 'Galveston'), ('Well 2', 'NY'), ('Well 2','LA'),
            ('Mobile', 'Galveston'), ('Mobile', 'NY'), ('Mobile','LA'),
            ('Galveston', 'Mobile'), ('Galveston', 'NY'), ('Galveston','LA'),
    ('NY', 'LA'), 
    ('LA', 'NY')
}
shippingCost = {  ('Well 1', 'Mobile'): 3530, ('Well 1', 'Galveston'): 3420, ('Well 1', 'NY'): 3920, ('Well 1','LA'): 3710,
            ('Well 2', 'Mobile'):1650 , ('Well 2', 'Galveston'):3580 , ('Well 2', 'NY'): 3840, ('Well 2','LA'):3620,
            ('Mobile', 'Galveston'): 1200, ('Mobile', 'NY'): 1680, ('Mobile','LA'): 3740,
            ('Galveston', 'Mobile'): 1500, ('Galveston', 'NY'): 3570, ('Galveston','LA'): 3650,
    ('NY', 'LA'): 4120, 
    ('LA', 'NY'): 3980

}

productionNodes = { 'Well 1', 'Well 2'}
transhipmentNodes = {'Mobile', 'Galveston'}
demandNodes = {'NY', 'LA'}


demand = { 'NY':160, 'LA': 140}
prodCapacity = {'Well 1':150, 'Well 2': 200}



In [3]:
model = pe.ConcreteModel()
model.V = pe.Set(initialize = nodes, ordered = False)
model.S = pe.Set(initialize = productionNodes, ordered = False)
model.T = pe.Set(initialize = transhipmentNodes, ordered = False)
model.D = pe.Set(initialize = demandNodes, ordered = False)
model.A = pe.Set(initialize = connections, ordered = False)

# decision variables
model.x = pe.Var(model.A, domain = pe.NonNegativeIntegers)

In [4]:
#Objective function
exprObj = sum( model.x[i,j]* shippingCost[i,j] for (i,j) in model.A)
model.obj = pe.Objective(expr = exprObj, sense = pe.minimize)
#print(exprObj)

In [5]:
# Production capacity constraints
model.flowBalanceS = pe.ConstraintList()
for i in model.S: 
    expression = sum(model.x[i,j] for j in model.V if((i,j) in model.A)) - \
    sum(model.x[j,i] for j in model.V if((j,i) in model.A)) <= prodCapacity[i]
    #print(expression)
    model.flowBalanceS.add(expression)

# Satisty the deamnd at each node
model.flowBalanceD = pe.ConstraintList()
for i in model.D: 
    expression = sum(model.x[j,i] for j in model.V if((j,i) in model.A)) - \
    sum(model.x[i,j] for j in model.V if((i,j) in model.A)) == demand[i]
    #print(expression)
    model.flowBalanceD.add(expression) 

# Net balance at each transhipment node
model.flowBalanceT = pe.ConstraintList()
for i in model.T:
    expression = sum(model.x[i,j] for j in model.V if((i,j) in model.A)) - \
    sum(model.x[j,i] for j in model.V if((j,i) in model.A)) == 0
    #print(expression)
    model.flowBalanceT.add(expression) 


In [6]:
# Next line of code allows you to print the model in .lp format. A file "OilDistribution.lp" will 
# be genrated. Open the file with an editor to see the model. Useful if you want to check it.

model.write(filename = "OilDistribution.lp", io_options = {"symbolic_solver_labels":True})

solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee= True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-12-10
Read LP format model from file C:\Users\rpo330\AppData\Local\Temp\tmpltuuleno.pyomo.lp
Reading time = 0.01 seconds
x17: 7 rows, 17 columns, 33 nonzeros
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 7 rows, 17 columns and 33 nonzeros
Model fingerprint: 0x6f8dc982
Variable types: 1 continuous, 16 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 1133200.0000
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 6 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (0 binary)
Found heuristic solution: objective 1133120.0000

Root relaxation: objective 1.048600e+06, 5 iterations, 0.00 seconds (0.0

In [7]:
print(result.solver.status)
print(result.solver.termination_condition)
print(pe.value(model.obj))
for (i,j) in model.A: 
    if pe.value(model.x[i,j]) > 0:
        print( str(model.x[i,j])+" = "+ str(pe.value(model.x[i,j])))

ok
optimal
1048600.0
x[Mobile,NY] = 160.0
x[Well 2,Mobile] = 160.0
x[Well 1,LA] = 100.0
x[Well 2,LA] = 40.0
